In [1]:
import os
os.chdir("..")

import pandas as pd

In [2]:
df = pd.read_excel(
    "data/ParamAndClfSelection.xlsx",
    engine="openpyxl",
    index_col="Unnamed: 0"
)

In [3]:
df.head()

,method,N,Appropriate (whole),Too bad (whole),Too good (whole),Appropriate (by sent),Too bad (by sent),Too good (by sent),Appropriate (raw mean by sent),Too bad (raw mean by sent),Too good (raw mean by sent):,N distractors
0,Baseline (no clf),3,0.5307,0.3333,0.1360,0.5307,0.3333,0.1360,1.5921,1.0000,0.4079,3.0000
1,XGBAllFeats_out,3,0.5812,0.2932,0.1257,0.5987,0.2763,0.1250,1.4605,0.7368,0.3158,2.5132
2,RandomForestFreqsOnly_out,3,0.5301,0.3388,0.1311,0.5000,0.3465,0.1272,1.2763,0.8158,0.3158,2.4079
3,CatBoostVecsOnly_out,3,0.5780,0.2948,0.1272,0.5746,0.2719,0.1272,1.3158,0.6711,0.2895,2.2763
4,CatBoostFeatDrop_out,3,0.5468,0.3153,0.1379,0.5504,0.3048,0.1447,1.4605,0.8421,0.3684,2.6711


In [4]:
def extract_model_name(s: str) -> str:
    if s.startswith("XGB"):
        return "XGBoost"
    elif s.startswith("Baseline"):
        return "Baseline"
    elif s.startswith("CatBoost"):
        return "CatBoost"
    elif s.startswith("RandomForest"):
        return "Random Forest"

def extract_setting(s: str) -> str:
    if "AllFeats" in s:
        return "AllFeats"
    elif "FreqsOnly" in s:
        return "FreqsOnly"
    elif "VecsOnly" in s:
        return "VecsOnly"
    elif "FeatDrop" in s:
        return "FeatDrop"
    else:
        return ""

def extract_drop_feat(s: str) -> str:
    if "FeatDrop" in s:
        return "Word2Vec vector of Ans"
    else:
        return "—"

def convert(df: pd.DataFrame) -> pd.DataFrame:
    df1 = pd.DataFrame()
    df1["Model"] = df["method"].apply(extract_model_name)
    df1["Setting"] = df["method"].apply(extract_setting)
    df1["Dropped feature"] = df["method"].apply(extract_drop_feat)
    df1["K"] = df["N"]
    df1["% Appropriate"] = df["Appropriate (by sent)"] * 100
    df1["% Too bad"] = df["Too bad (by sent)"] * 100
    df1["% Too good"] = df["Too good (by sent)"] * 100
    df1["Appropriate (raw mean by sent)"] = df["Appropriate (raw mean by sent)"]
    df1["N distractors"] = df["N distractors"]
    df1 = df1.sort_values(by="% Appropriate", ascending=False)
    return df1

In [5]:
df = convert(df)

In [6]:
df

,Model,Setting,Dropped feature,K,% Appropriate,% Too bad,% Too good,Appropriate (raw mean by sent),N distractors
1,XGBoost,AllFeats,—,3,59.87,27.63,12.50,1.4605,2.5132
3,CatBoost,VecsOnly,—,3,57.46,27.19,12.72,1.3158,2.2763
6,XGBoost,AllFeats,—,4,56.14,32.68,11.18,1.7368,3.2368
4,CatBoost,FeatDrop,Word2Vec vector of Ans,3,55.04,30.48,14.47,1.4605,2.6711
8,CatBoost,VecsOnly,—,4,53.62,31.47,12.28,1.5789,2.9079
...,...,...,...,...,...,...,...,...,...
70,Baseline,,—,17,29.88,59.75,10.37,5.0789,17.0000
87,Random Forest,FreqsOnly,—,20,29.66,61.08,9.26,4.2105,14.3816
75,Baseline,,—,18,28.73,61.11,10.16,5.1711,18.0000
80,Baseline,,—,19,27.98,62.12,9.90,5.3158,19.0000


In [7]:
df.reset_index().to_excel("full_tables/Table2.xlsx", engine="openpyxl")
df.reset_index().to_csv("full_tables/Table2.csv")